In [ ]:
#### without Pre-Process Train and Validation data

In [1]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
#from keras.applications.vgg19 import preprocess_input                
#from keras.applications.resnet50 import preprocess_input            
#from keras.applications.inception_resnet_v2 import preprocess_input               
#from keras.applications.mobilenet import preprocess_input               
from keras.applications.densenet import preprocess_input     


from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

#from keras.applications.xception import Xception

#from keras.applications.inception_v3 import InceptionV3

#from keras.applications.vgg19 import VGG19

#from keras.applications.resnet50 import ResNet50   

#from keras.applications.inception_resnet_v2 import InceptionResNetV2    

#from keras.applications.mobilenet import MobileNet 

from keras.applications.densenet import DenseNet201    

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_DenseNet201_data4.h5'  
train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception_ResNet_V2 network

    model = applications.DenseNet201(weights='imagenet', include_top=False)

    model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_DenseNet201_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_DenseNet201_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_DenseNet201_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
    
    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_DenseNet201_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_validation shape',y_pred_validation.shape)
    print(' y_pred_validation : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

validation loss: 2.9046404780447483
validation accuracy: 0.8175
 y_pred_validation shape (400,)
 y_pred_validation : 
  [1.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 1.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.6015804e-08 1.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 1.1517097e-36 0.0000000e+00 9.9931860e-01 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 1.3166234e-11 1.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
 2.7768221e-26 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00

In [ ]:
#### Pre-Process Train data

In [3]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
#from keras.applications.vgg19 import preprocess_input                
#from keras.applications.resnet50 import preprocess_input            
#from keras.applications.inception_resnet_v2 import preprocess_input               
#from keras.applications.mobilenet import preprocess_input               
from keras.applications.densenet import preprocess_input     


from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

#from keras.applications.xception import Xception

#from keras.applications.inception_v3 import InceptionV3

#from keras.applications.vgg19 import VGG19

#from keras.applications.resnet50 import ResNet50   

#from keras.applications.inception_resnet_v2 import InceptionResNetV2    

#from keras.applications.mobilenet import MobileNet 

from keras.applications.densenet import DenseNet201    

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_DenseNet201_Pre_Process_TrainData_data4.h5'  
train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception_ResNet_V2 network

    model = applications.DenseNet201(weights='imagenet', include_top=False)

    model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_DenseNet201_Pre_Process_TrainData_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_DenseNet201_Pre_Process_TrainData_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_DenseNet201_Pre_Process_TrainData_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    preprocess_input(train_data, data_format = 'channels_last')   
    
    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_DenseNet201_Pre_Process_TrainData_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_validation shape',y_pred_validation.shape)
    print(' y_pred_validation : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

Epoch 39/50
1000/1000 [==============================] - 31s 31ms/step - loss: 0.1149 - acc: 0.9620 - val_loss: 2.1576 - val_acc: 0.8650
Epoch 40/50
1000/1000 [==============================] - 32s 32ms/step - loss: 0.1289 - acc: 0.9450 - val_loss: 2.2826 - val_acc: 0.8550
Epoch 41/50
1000/1000 [==============================] - 31s 31ms/step - loss: 0.1406 - acc: 0.9420 - val_loss: 1.9915 - val_acc: 0.8750
Epoch 42/50
1000/1000 [==============================] - 31s 31ms/step - loss: 0.1210 - acc: 0.9570 - val_loss: 2.0244 - val_acc: 0.8725
Epoch 43/50
1000/1000 [==============================] - 31s 31ms/step - loss: 0.1229 - acc: 0.9460 - val_loss: 2.2562 - val_acc: 0.8575
Epoch 44/50
1000/1000 [==============================] - 31s 31ms/step - loss: 0.1223 - acc: 0.9600 - val_loss: 2.0507 - val_acc: 0.8725
Epoch 45/50
1000/1000 [==============================] - 31s 31ms/step - loss: 0.1115 - acc: 0.9570 - val_loss: 2.1321 - val_acc: 0.8675
Epoch 46/50
1000/1000 [==================

validation loss: 1.9678775962720976
validation accuracy: 0.8775
 y_pred_validation shape (400,)
 y_pred_validation : 
  [0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00

In [ ]:
#### Pre-Process Train data and Validation data

In [4]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
#from keras.applications.vgg19 import preprocess_input                
#from keras.applications.resnet50 import preprocess_input            
#from keras.applications.inception_resnet_v2 import preprocess_input               
#from keras.applications.mobilenet import preprocess_input               
from keras.applications.densenet import preprocess_input     


from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

#from keras.applications.xception import Xception

#from keras.applications.inception_v3 import InceptionV3

#from keras.applications.vgg19 import VGG19

#from keras.applications.resnet50 import ResNet50   

#from keras.applications.inception_resnet_v2 import InceptionResNetV2    

#from keras.applications.mobilenet import MobileNet 

from keras.applications.densenet import DenseNet201    

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_DenseNet201_Pre_Process_data4.h5'  
train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception_ResNet_V2 network

    model = applications.DenseNet201(weights='imagenet', include_top=False)

    #model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_DenseNet201_Pre_Process_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_DenseNet201_Pre_Process_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_DenseNet201_Pre_Process_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    preprocess_input(train_data, data_format = 'channels_last')   
    
    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_DenseNet201_Pre_Process_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    preprocess_input(validation_data, data_format = 'channels_last')   
    
    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_validation shape',y_pred_validation.shape)
    print(' y_pred_validation : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

validation loss: 0.4041490649059415
validation accuracy: 0.86
 y_pred_validation shape (400,)
 y_pred_validation : 
  [2.78481156e-01 9.72565174e-01 8.01271021e-01 8.75750482e-01
 3.86885330e-02 7.33184442e-03 3.85832973e-03 2.63671935e-01
 3.43447900e-03 1.93725049e-03 7.25557446e-01 9.29442585e-01
 1.04777433e-03 2.15911232e-02 2.73001269e-02 9.58229542e-01
 9.43368077e-01 9.99834180e-01 9.53434467e-01 7.30429471e-01
 1.21006876e-01 2.74438024e-01 3.89423370e-01 4.92663443e-01
 1.55933857e-01 1.06109641e-01 1.92585558e-01 8.88611615e-01
 5.86061412e-03 2.65829079e-02 2.92435259e-01 3.98885022e-04
 2.54750282e-01 4.49767970e-02 5.74488484e-04 9.18203890e-02
 1.82759925e-03 2.23177049e-04 3.31031919e-01 7.27830529e-01
 1.15043484e-03 2.23960429e-02 2.12352094e-03 2.31738988e-04
 5.67015171e-01 8.20724428e-01 9.73417938e-01 3.89905661e-01
 3.26491287e-03 1.87063031e-02 2.32736364e-01 2.41911352e-01
 5.29382331e-03 1.31737422e-02 1.02241070e-03 2.37328536e-03
 2.04732224e-01 1.13098770e-